In [1]:
import sys
import pandas as pd
from metrics import get_scores


In [2]:
%load_ext memory_profiler

In [3]:
sys.path.append('../archive/src')
#sys.path

In [4]:
from predictor import ScoringService

In [5]:
inputs = ScoringService.get_inputs("../input/")

In [6]:
ScoringService.get_model("../archive/model/")

True

In [7]:
#%%mprun -f ScoringService.get_dataset
dfs = ScoringService.get_dataset(inputs)

Mem. usage decreased to 217.60 Mb (43.8% reduction)
Mem. usage decreased to  0.29 Mb (21.9% reduction)
Mem. usage decreased to 20.13 Mb (28.3% reduction)
Mem. usage decreased to 306.26 Mb (44.1% reduction)


In [8]:
#%%mprun -f ScoringService._predict

sb_df = ScoringService._predict(dfs)
sb_df.shape

(80663, 3)

In [9]:
sb_df.tail()

,,ID,high_20,low_20
base_date,Local Code,,,
2020-01-31,9997,2020-01-31-9997,-0.111284,0.327301
2020-05-13,9997,2020-05-13-9997,0.496445,-1.584168
2020-07-31,9997,2020-07-31-9997,0.490590,-0.189178
2020-10-29,9997,2020-10-29-9997,0.045150,-1.102403
2020-10-30,9997,2020-10-30-9997,0.953869,-0.696546


In [10]:
sb_df.isna().sum()

ID         0
high_20    0
low_20     0
dtype: int64

In [11]:
import hashlib
from datetime import datetime
time_str = datetime.now().strftime('%y%m%d%H%M%S')
hash_str = hashlib.md5(
    pd.util.hash_pandas_object(sb_df).values).hexdigest()[:8]
hash_str

'89db02a9'

In [12]:
lb_df = dfs["stock_labels"]
ls_df = dfs["stock_list"]
fn_df = dfs["stock_fin"]
trg_codes = ls_df.loc[ls_df.prediction_target, "Local Code"]

ty_df = lb_df.loc[lb_df["Local Code"].isin(trg_codes), ["base_date", "Local Code", "label_high_20", "label_low_20"]].copy()
last_fn_df = fn_df.groupby(["Local Code", "Result_FinancialStatement FiscalYear", "Result_FinancialStatement ReportType"]).nth(-1)
ty_df = ty_df.merge(last_fn_df, on=["Local Code", "base_date"], how="inner")

ty_df["ID"] = ty_df.base_date.dt.strftime("%Y-%m-%d-") + ty_df["Local Code"].astype("str")
ty_df = ty_df.merge(sb_df, on="ID", how="inner")

ty_df.dropna(subset=["label_high_20", "label_low_20"], inplace=True)
#ty_df.fillna(0., inplace=True)
ty_df

,base_date,Local Code,label_high_20,label_low_20,Result_FinancialStatement AccountingStandard,Result_FinancialStatement FiscalPeriodEnd,Result_FinancialStatement ModifyDate,Result_FinancialStatement CompanyType,Result_FinancialStatement ChangeOfFiscalYearEnd,Result_FinancialStatement NetSales,...,Forecast_Dividend FiscalPeriodEnd,Forecast_Dividend ReportType,Forecast_Dividend FiscalYear,Forecast_Dividend ModifyDate,Forecast_Dividend RecordDate,Forecast_Dividend QuarterlyDividendPerShare,Forecast_Dividend AnnualDividendPerShare,ID,high_20,low_20
0,2016-02-05,1301,0.030197,-0.056610,ConsolidatedJP,2015/12,2016-02-05,GB,False,178890.0,...,2016/03,Annual,2016.0,2016/02/05,2016-03-31,5.00,5.0,2016-02-05-1301,-0.411177,0.645418
1,2016-05-09,1301,0.034485,-0.011490,ConsolidatedJP,2016/03,2016-05-09,GB,False,226626.0,...,2017/03,Annual,2017.0,2016/05/09,2017-03-31,50.00,50.0,2016-05-09-1301,-0.845245,0.442832
2,2016-08-05,1301,0.026917,0.000000,ConsolidatedJP,2016/06,2016-08-05,GB,False,52206.0,...,2017/03,Annual,2017.0,2016/08/05,2017-03-31,50.00,50.0,2016-08-05-1301,-0.948111,1.334037
3,2016-11-04,1301,0.007790,-0.039673,ConsolidatedJP,2016/09,2016-11-04,GB,False,109570.0,...,2017/03,Annual,2017.0,2016/11/04,2017-03-31,50.00,50.0,2016-11-04-1301,-0.835821,1.406967
4,2017-02-17,1301,0.135864,0.001770,ConsolidatedJP,2016/12,2017-02-10,GB,False,179975.0,...,2017/03,Annual,2017.0,2017/02/17,2017-03-31,60.00,60.0,2017-02-17-1301,0.144979,2.030668
5,2017-05-11,1301,0.044403,-0.026321,ConsolidatedJP,2017/03,2017-05-11,GB,False,236561.0,...,2018/03,Annual,2018.0,2017/05/11,2018-03-31,50.00,50.0,2017-05-11-1301,-1.046527,-0.189704
6,2017-08-04,1301,0.055908,-0.015533,ConsolidatedJP,2017/06,2017-08-04,GB,False,56844.0,...,2018/03,Annual,2018.0,2017/08/04,2018-03-31,50.00,50.0,2017-08-04-1301,-0.228726,0.655288
7,2017-11-06,1301,0.022079,-0.087036,ConsolidatedJP,2017/09,2017-11-06,GB,False,120458.0,...,2018/03,Annual,2018.0,2017/11/06,2018-03-31,50.00,50.0,2017-11-06-1301,-0.217253,-0.481006
8,2018-02-09,1301,0.038055,-0.043304,ConsolidatedJP,2017/12,2018-02-09,GB,False,198323.0,...,2018/03,Annual,2018.0,2018/02/09,2018-03-31,60.00,60.0,2018-02-09-1301,-0.538758,-0.137890
9,2018-05-10,1301,0.010483,-0.034088,ConsolidatedJP,2018/03,2018-05-10,GB,False,254783.0,...,2019/03,Annual,2019.0,2018/05/10,2019-03-31,60.00,60.0,2018-05-10-1301,-1.846101,-0.528149


In [13]:
sub_ty_df = ty_df[("2020-01-01" <= ty_df.base_date) & (ty_df.base_date <= "2020-11-30")]
get_scores(sub_ty_df[["label_high_20", "label_low_20"]].values, sub_ty_df[["high_20", "low_20"]].values)

(0.6835267145826136, 0.45047776401008494, 0.40213002823023036)

In [14]:
tb_df = ScoringService.get_table(dfs) #.dropna(subset=["label_high_20", "label_low_20"])
tb_df.shape

(80663, 183)

In [15]:
tb_df.dropna(subset=["label_high_20", "label_low_20"]).shape

(80321, 183)

In [16]:
sb_df[sb_df.ID.isin(tb_df.ID)].shape

(80663, 3)

In [17]:
sb_df.shape

(80663, 3)

In [18]:
tb_df[tb_df["Local Code"] == 9997]

,base_date,Local Code,label_date_5,label_high_5,label_low_5,label_date_10,label_high_10,label_low_10,label_date_20,label_high_20,...,feat_pbr,feat_roe,feat_ma_roe,feat_fc_roe,feat_dv_fill_Result_Dividend AnnualDividendPerShare,feat_amv_ratio_trading_value,feat_amv_ratio_Result_FinancialStatement NetAssets,feat_amv_ratio_Result_FinancialStatement TotalAssets,feat_dprfill_Result_Dividend AnnualDividendPerShare,ID
80637,2016-01-29,9997,2016-02-05,-0.008179,-0.055664,2016-02-15,-0.008179,-0.140747,2016-02-29,-0.008179,...,7.224402e+05,0.018603,0.018603,0.020518,NaN,0.000812,1.384198e-06,0.000003,NaN,2016-01-29-9997
80638,2016-05-10,9997,2016-05-17,0.129761,-0.045624,2016-05-24,0.129761,-0.045624,2016-06-07,0.129761,...,6.739622e+05,0.018603,0.018603,0.010773,NaN,0.000735,1.483763e-06,0.000003,NaN,2016-05-10-9997
80639,2016-05-11,9997,2016-05-18,0.183838,0.066162,2016-05-25,0.183838,0.066162,2016-06-08,0.183838,...,6.559654e+05,-0.012970,0.003001,0.015500,0.022978,0.001201,1.524470e-06,0.000003,-1.162099e+06,2016-05-11-9997
80640,2016-07-29,9997,2016-08-05,0.003010,-0.093384,2016-08-15,0.058716,-0.093384,2016-08-29,0.058716,...,8.294411e+05,-0.013925,-0.002569,0.016057,0.018825,0.000781,1.205631e-06,0.000003,-1.121361e+06,2016-07-29-9997
80641,2016-10-31,9997,2016-11-08,-0.038513,-0.098450,2016-11-15,-0.038513,-0.108398,2016-11-30,-0.038513,...,8.761664e+05,0.003303,-0.001102,0.022493,0.017832,0.000879,1.141336e-06,0.000003,4.729789e+06,2016-10-31-9997
80642,2017-01-31,9997,2017-02-07,0.109680,0.007740,2017-02-14,0.109680,0.007740,2017-02-28,0.130371,...,8.758829e+05,0.083074,0.015326,0.020338,0.016129,0.001243,1.141705e-06,0.000002,1.700554e+05,2017-01-31-9997
80643,2017-05-11,9997,2017-05-18,0.127930,0.042053,2017-05-25,0.166382,0.042053,2017-06-08,0.229370,...,1.212052e+06,-0.005913,0.016526,0.016520,0.011426,0.002801,8.250470e-07,0.000002,-2.342111e+06,2017-05-11-9997
80644,2017-07-31,9997,2017-08-07,-0.007881,-0.085876,2017-08-15,-0.007881,-0.120544,2017-08-29,-0.007881,...,1.381400e+06,0.034758,0.027960,0.016232,0.009850,0.002008,7.239033e-07,0.000001,3.914834e+05,2017-07-31-9997
80645,2017-10-30,9997,2017-11-07,-0.000770,-0.068787,2017-11-14,-0.000770,-0.068787,2017-11-29,-0.000770,...,1.408614e+06,0.034758,0.027960,0.024235,0.009660,0.008443,7.099175e-07,0.000001,3.914834e+05,2017-10-30-9997
80646,2017-10-31,9997,2017-11-08,-0.002340,-0.061523,2017-11-15,-0.002340,-0.073975,2017-11-30,-0.002340,...,1.390677e+06,0.013677,0.030523,0.024503,0.009735,0.004749,7.190744e-07,0.000002,9.898663e+05,2017-10-31-9997


In [19]:
tb_df.ID.value_counts()

2018-05-15-6307    1
2016-08-19-6327    1
2018-05-11-3205    1
2016-01-08-9313    1
2019-02-13-8078    1
2020-07-22-4684    1
2018-04-27-5959    1
2018-08-07-2345    1
2020-08-07-6180    1
2018-01-31-6407    1
2018-11-06-2531    1
2018-04-13-7608    1
2019-11-07-8037    1
2018-08-15-4832    1
2019-11-05-3082    1
2018-02-13-3807    1
2018-10-17-3612    1
2020-06-10-8142    1
2018-05-15-6463    1
2018-05-11-3905    1
2016-10-03-7603    1
2017-10-27-6419    1
2017-06-15-7571    1
2019-02-07-3937    1
2019-05-15-5930    1
2020-05-15-8363    1
2019-02-13-7758    1
2019-11-14-6357    1
2018-07-31-5820    1
2019-11-13-4978    1
                  ..
2018-04-27-6180    1
2016-07-28-6161    1
2019-08-08-5391    1
2018-04-27-2175    1
2018-05-02-9854    1
2019-07-31-8303    1
2018-02-08-3566    1
2019-11-08-1795    1
2017-08-08-1926    1
2016-01-28-8706    1
2019-01-11-3093    1
2018-05-18-1981    1
2020-10-30-2413    1
2016-02-12-3096    1
2019-10-29-8860    1
2018-10-11-3922    1
2019-11-11-76

In [20]:
tb_df.quarter_span.value_counts()

3.0     80324
6.0        55
9.0        14
2.0        13
4.0        11
7.0         5
5.0         4
12.0        2
1.0         1
8.0         1
Name: quarter_span, dtype: int64

In [21]:
sb_df[sb_df.index.get_level_values("Local Code") == 3139]

,,ID,high_20,low_20
base_date,Local Code,,,
2016-01-14,3139,2016-01-14-3139,0.877229,-0.156158
2016-04-12,3139,2016-04-12-3139,0.334404,0.092925
2016-07-12,3139,2016-07-12-3139,0.124421,-0.203704
2016-10-12,3139,2016-10-12-3139,0.456369,0.380244
2017-01-05,3139,2017-01-05-3139,1.053447,1.214721
2017-01-12,3139,2017-01-12-3139,1.008656,0.155285
2017-04-12,3139,2017-04-12-3139,1.704330,2.381305
2017-07-05,3139,2017-07-05-3139,1.858920,-0.612867
2017-07-12,3139,2017-07-12-3139,1.105609,-1.649780


In [22]:
sb_df

,,ID,high_20,low_20
base_date,Local Code,,,
2016-02-05,1301,2016-02-05-1301,-0.411177,0.645418
2016-05-09,1301,2016-05-09-1301,-0.845245,0.442832
2016-08-05,1301,2016-08-05-1301,-0.948111,1.334037
2016-11-04,1301,2016-11-04-1301,-0.835821,1.406967
2017-02-10,1301,2017-02-10-1301,0.011451,2.271785
2017-02-17,1301,2017-02-17-1301,0.144979,2.030668
2017-05-11,1301,2017-05-11-1301,-1.046527,-0.189704
2017-08-04,1301,2017-08-04-1301,-0.228726,0.655288
2017-11-06,1301,2017-11-06-1301,-0.217253,-0.481006


In [23]:
tb_df[tb_df["Local Code"] == 3139]

,base_date,Local Code,label_date_5,label_high_5,label_low_5,label_date_10,label_high_10,label_low_10,label_date_20,label_high_20,...,feat_pbr,feat_roe,feat_ma_roe,feat_fc_roe,feat_dv_fill_Result_Dividend AnnualDividendPerShare,feat_amv_ratio_trading_value,feat_amv_ratio_Result_FinancialStatement NetAssets,feat_amv_ratio_Result_FinancialStatement TotalAssets,feat_dprfill_Result_Dividend AnnualDividendPerShare,ID
13826,2016-01-14,3139,2016-01-21,-0.151123,-0.323486,2016-01-28,-0.151123,-0.323486,2016-02-12,-0.151123,...,7.643185e+05,0.019995,0.019995,0.013474,0.037313,0.007201,1.308355e-06,0.000005,1.426310e+06,2016-01-14-3139
13827,2016-04-12,3139,2016-04-19,0.036011,-0.008003,2016-04-26,0.036011,-0.008003,2016-05-16,0.036011,...,6.266622e+05,0.016039,0.018564,0.014212,0.048000,0.001300,1.595756e-06,0.000007,1.875418e+06,2016-04-12-3139
13828,2016-07-12,3139,2016-07-20,-0.018646,-0.067627,2016-07-27,-0.002331,-0.067627,2016-08-10,-0.002331,...,6.283716e+05,0.011073,0.015744,0.017301,0.046620,0.003691,1.591415e-06,0.000006,2.645679e+06,2016-07-12-3139
13829,2016-10-12,3139,2016-10-19,0.015686,-0.048462,2016-10-26,0.019241,-0.048462,2016-11-10,0.054169,...,6.873864e+05,0.022321,0.017429,0.017361,0.042766,0.001356,1.454786e-06,0.000005,1.316960e+06,2016-10-12-3139
13830,2017-01-05,3139,2017-01-13,0.134888,0.043518,2017-01-20,0.150024,0.043518,2017-02-03,0.329834,...,7.770456e+05,0.022321,0.017429,0.023462,0.037831,0.003259,1.286926e-06,0.000005,1.316960e+06,2017-01-05-3139
13831,2017-01-12,3139,2017-01-19,0.047668,-0.033905,2017-01-26,0.078125,-0.033905,2017-02-09,0.211426,...,7.529646e+05,0.039496,0.020711,0.017252,0.036760,0.009569,1.328084e-06,0.000004,7.008213e+05,2017-01-12-3139
13832,2017-04-12,3139,2017-04-19,0.231201,0.045624,2017-04-26,0.381836,0.045624,2017-05-15,0.567871,...,7.943581e+05,0.072284,0.034687,0.016616,0.033561,0.002589,1.258878e-06,0.000004,3.688103e+05,2017-04-12-3139
13833,2017-07-05,3139,2017-07-12,0.086731,-0.011833,2017-07-20,0.086731,-0.122192,2017-08-03,0.224731,...,1.584967e+06,0.072284,0.034687,0.050607,0.016820,0.004080,6.309278e-07,0.000002,3.688103e+05,2017-07-05-3139
13834,2017-07-12,3139,2017-07-20,-0.033112,-0.149048,2017-07-27,0.187256,-0.149048,2017-08-10,0.187256,...,1.585344e+06,0.028053,0.038358,0.028216,0.016306,0.011417,6.307778e-07,0.000002,9.214880e+05,2017-07-12-3139
13835,2017-10-12,3139,2017-10-19,-0.034912,-0.107117,2017-10-26,-0.032593,-0.112915,2017-11-10,-0.005821,...,1.685442e+06,0.029638,0.040207,0.027414,0.014901,0.040271,5.933162e-07,0.000002,8.473737e+05,2017-10-12-3139


In [24]:
sb_df.to_csv("../tmp/submission.csv", header=False, index=False)